# Kaggle Healthcare Dataset — FAIR Reproducible Workflow

This notebook demonstrates a small, **reproducible** analysis using a healthcare CSV from Kaggle (uploader: *prasad22*).  
We load the data, do quick EDA, save outputs, and reflect on **FAIR** principles (Findable, Accessible, Interoperable, Reusable).

In [9]:
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reproducibility
np.random.seed(42)

# Paths
DATA_PATH = "dataset/healthcare.csv"
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

pd.options.display.max_columns = 50
pd.options.display.width = 120

ModuleNotFoundError: No module named 'pandas'

In [2]:
pd.read_csv("dataset/healthcare.csv")
df = pd.read_csv(DATA_PATH)


NameError: name 'pd' is not defined

In [3]:
# Load CSV 
df_raw = pd.read_csv(DATA_PATH)
df_raw.head()

NameError: name 'pd' is not defined

In [ ]:
print("Shape:", df_raw.shape)
print("\nColumn names:", list(df_raw.columns))

# Missing values per column
na_summary = df_raw.isna().sum().sort_values(ascending=False)
print("\nMissing values per column:\n", na_summary)

# Dtypes overview
print("\nDtypes:\n", df_raw.dtypes)

In [ ]:
df = df_raw.copy()

# Standardize column names: lower + snake_case
def to_snake(name: str) -> str:
    return (
        name.strip()
            .replace("/", " ")
            .replace("-", " ")
            .replace("(", " ")
            .replace(")", " ")
            .replace("%", "pct")
            .replace("#", "n")
    )

df.columns = (
    df.columns
      .str.replace(r"[\s]+", "_", regex=True)
      .map(to_snake)
      .str.lower()
)

# Try to find commonly expected columns (be robust to naming)
def find_col(candidates):
    cols = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in cols:
            return cols[key]
    return None

age_col     = find_col(["age"])
gender_col  = find_col(["gender","sex"])
cond_col    = find_col(["medical_condition","condition","diagnosis"])
date_col    = find_col(["date_of_admission","admission_date","date"])
cost_col    = find_col(["treatment_cost","billing_amount","charges","charge","cost"])

# Parse date if present
if date_col is not None:
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# Make age numeric if present
if age_col is not None:
    df[age_col] = pd.to_numeric(df[age_col], errors="coerce")

# Make cost numeric if present
if cost_col is not None:
    df[cost_col] = (
        pd.to_numeric(df[cost_col], errors="coerce")
        .astype("float64")
    )

# Quick preview after cleaning
df.head()

In [ ]:
summary = {
    "n_rows": len(df),
    "n_cols": df.shape[1],
    "n_missing_cells": int(df.isna().sum().sum()),
    "pct_rows_with_any_na": float((df.isna().any(axis=1)).mean()*100),
}
display(pd.DataFrame([summary]))

# Numeric summary
display(df.describe(datetime_is_numeric=True, include="all").T.iloc[:15])

#Simple EDA tables (Code)

In [ ]:
tables_to_save = {}

# Gender counts
if gender_col is not None and gender_col in df:
    gender_counts = df[gender_col].value_counts(dropna=False).rename_axis(gender_col).to_frame("count")
    tables_to_save["gender_counts.csv"] = gender_counts

# Top medical conditions
if cond_col is not None and cond_col in df:
    top_conditions = df[cond_col].value_counts().head(10).rename_axis(cond_col).to_frame("count")
    tables_to_save["top10_conditions.csv"] = top_conditions

# Average cost by gender
if cost_col is not None and gender_col is not None and all(c in df for c in [cost_col, gender_col]):
    avg_cost_gender = df.groupby(gender_col, dropna=False)[cost_col].mean().to_frame("avg_cost").sort_values("avg_cost", ascending=False)
    tables_to_save["avg_cost_by_gender.csv"] = avg_cost_gender

# Events per month (if admission date exists)
if date_col is not None and date_col in df:
    per_month = df.set_index(date_col).resample("MS").size().rename("n_admissions").to_frame()
    tables_to_save["admissions_per_month.csv"] = per_month

# Save all tables
for fname, table in tables_to_save.items():
    out_path = os.path.join(OUT_DIR, fname)
    table.to_csv(out_path)
    print("Saved:", out_path)

# Show one or two in the output
next(iter(tables_to_save.values())) if tables_to_save else df.head(3)

In [ ]:
# 1) Age histogram
if age_col is not None and age_col in df:
    plt.figure()
    df[age_col].dropna().astype(float).plot(kind="hist", bins=20)
    plt.title("Age Distribution")
    plt.xlabel("Age")
    plt.ylabel("Count")
    fig1_path = os.path.join(OUT_DIR, "age_hist.png")
    plt.savefig(fig1_path, bbox_inches="tight", dpi=150)
    plt.show()
    print("Saved:", fig1_path)

# 2) Cost by gender (boxplot)
if cost_col is not None and gender_col is not None and all(c in df for c in [cost_col, gender_col]):
    plt.figure()
    # Prepare data in simple lists by gender categories
    grouped = [g.dropna().values for _, g in df.groupby(gender_col)[cost_col]]
    labels  = [str(k) for k, _ in df.groupby(gender_col)[cost_col]]
    plt.boxplot(grouped, labels=labels, showfliers=False)
    plt.title("Treatment Cost by Gender")
    plt.xlabel("Gender")
    plt.ylabel("Cost")
    fig2_path = os.path.join(OUT_DIR, "cost_by_gender_box.png")
    plt.savefig(fig2_path, bbox_inches="tight", dpi=150)
    plt.show()
    print("Saved:", fig2_path)

# 3) Top conditions bar chart
if cond_col is not None and cond_col in df:
    top = df[cond_col].value_counts().head(10)
    plt.figure()
    top.plot(kind="bar")
    plt.title("Top 10 Medical Conditions")
    plt.xlabel("Condition")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha="right")
    fig3_path = os.path.join(OUT_DIR, "top10_conditions_bar.png")
    plt.savefig(fig3_path, bbox_inches="tight", dpi=150)
    plt.show()
    print("Saved:", fig3_path)

# 4) Admissions per month
if date_col is not None and date_col in df:
    per_month = df.set_index(date_col).resample("MS").size()
    if len(per_month) > 0:
        plt.figure()
        per_month.plot()
        plt.title("Admissions per Month")
        plt.xlabel("Month")
        plt.ylabel("Count")
        fig4_path = os.path.join(OUT_DIR, "admissions_per_month.png")
        plt.savefig(fig4_path, bbox_inches="tight", dpi=150)
        plt.show()
        print("Saved:", fig4_path)

In [ ]:
# Create a light, privacy-aware subset (no names/identifiers if present)
cols_to_keep = []
for c in [age_col, gender_col, cond_col, date_col, cost_col]:
    if c is not None and c in df.columns:
        cols_to_keep.append(c)

subset = df[cols_to_keep].copy()
clean_path = os.path.join(OUT_DIR, "healthcare_clean_subset.csv")
subset.to_csv(clean_path, index=False)
print("Saved:", clean_path)
subset.head()

## FAIR discussion

**Findable** — This workflow is publicly hosted on GitHub with clear metadata (README, keywords).  
The original dataset comes from Kaggle (uploader: *prasad22*). Kaggle datasets do **not** provide a DOI; to improve
findability and citability, this project can be archived on **Zenodo** (via a GitHub release), which will mint a DOI.

**Accessible** — Data is included as a CSV and loaded with pandas. The repository is public. The provenance (Kaggle link) is documented.

**Interoperable** — Open, widely used formats are used (CSV for data, PNG/CSV for outputs). Column names are standardized
to snake_case. Versioned dependencies are given in `environment.yml`.

**Reusable** — The code is licensed (MIT) and the dataset attribution is provided. The notebook documents context, assumptions,
and a small cleaned subset is exported. A risk assessment is included below.

## Risk assessment

- **Upstream availability**: Kaggle datasets may be updated or removed.  
  *Mitigation*: include the CSV in the repo and optionally archive the repo to Zenodo for a DOI.
- **Package changes**: future updates (e.g., pandas 3.x) might change parsing defaults.  
  *Mitigation*: versions pinned in `environment.yml`.
- **Data quality**: missing or inconsistent values could bias results.  
  *Mitigation*: basic cleaning/validation, explicit dtype conversions, and clear reporting of missingness.

In [ ]:
print("Artifacts created in:", OUT_DIR)
for f in sorted(os.listdir(OUT_DIR)):
    print(" -", f)

##Basic summary (head, describe, null counts)

##Simple plots (histogram of Age, bar chart of Gender, etc.)

##Save outputs (to outputs/)

##FAIR discussion (Markdown cell)